In [1]:
# linking Finrl environment with trading API
# After reading this article you will be able to
# Train a DRL agent on minute level data of Dow 30
# Deploy the DRL agent to Alpaca trading API using FinRL
# Place trades on Alpaca using virtual money through Jupyter Notebook
# https://medium.datadriveninvestor.com/a-data-scientists-approach-for-algorithmic-trading-using-deep-reinforcement-learning-an-be8da40b2230

In [2]:
## install finrl library
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-uxgcw0ys
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-uxgcw0ys
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-_kz4rey7/pyfolio_a25b6b613afc4ad69262c5fb85107218
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-_kz4rey7/pyfolio_a25b6b613afc4ad69262c5fb85107218
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-_kz4rey7/elegantrl_e83d740ca8dc4942bdd4534bdd69e6d0
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-_kz4rey7/elegantrl_e83d740ca8dc4942bdd4534bdd69e6d0
     |████████████████████████████████| 234 kB 8.2 MB/s 
     |████████████████████████████████| 177 kB 42.1 MB/s 
     |████████████████████████████████| 52.7 MB 163 kB/s 
     |████████████████████████████████| 1.2 MB 36.9 MB/s 
     |█████

In [9]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

In [10]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [3]:
# link for alpaca ....SDk(software development kit ) with python
##   https://alpaca.markets/docs/clients/


In [4]:
## github link for alpaca trading API
## https://github.com/alpacahq/alpaca-trade-api-python/

In [5]:
# library for alpaca trading API
! pip3 install alpaca-trade-api

In [6]:
## Training-Testing-Trading Pipeline
## We use historical data (time series) for the “training-testing” part, which is the same as conventional machine learning tasks, and this testing period is for backtesting purpose.
## For the “trading” part, we use live trading APIs, in this case is Alpaca, allowing users carry out trades directly in a trading system.

In [7]:
# services provided by ALPACA are for both historical and live stream data


In [14]:
# Pick a data source
DP = DataProcessor(data_source = 'alpaca',
                   API_KEY = "PK49CIAXU93FSNDK72ER",
                   API_SECRET = "VTPojIlhzmXC98wFEGozlfJibNUTava6g6cAyglG",
                   APCA_API_BASE_URL = "https://paper-api.alpaca.markets")

Alpaca successfully connected


In [15]:
# I Am Getting the stock ticker list, set start/end date, and specify the time_interval
data = DP.download_data(start_date = '2021-10-01',
                        end_date = '2021-10-05',
                        ticker_list ="AAPL",
                        time_interval= '1Min')

Data before 2021-10-01T15:59:00-04:00 is successfully fetched
Data before 2021-10-02T15:59:00-04:00 is successfully fetched
Data before 2021-10-03T15:59:00-04:00 is successfully fetched
Data before 2021-10-04T15:59:00-04:00 is successfully fetched
Data before 2021-10-05T15:59:00-04:00 is successfully fetched


In [16]:
data

,timestamp,open,high,low,close,volume,trade_count,vwap,tic,index,time
0,2021-10-01 13:30:00+00:00,157.15,157.50,157.13,157.480,27671.0,86.0,157.159983,A,NaN,2021-10-01 13:30:00
1,2021-10-01 13:31:00+00:00,157.14,157.39,156.57,157.185,10115.0,178.0,156.884904,A,NaN,2021-10-01 13:31:00
2,2021-10-01 13:32:00+00:00,157.16,157.47,156.48,156.480,2674.0,69.0,156.874718,A,NaN,2021-10-01 13:32:00
3,2021-10-01 13:33:00+00:00,156.20,156.20,155.94,155.940,6238.0,120.0,156.032277,A,NaN,2021-10-01 13:33:00
4,2021-10-01 13:34:00+00:00,155.79,155.79,155.34,155.415,3649.0,100.0,155.585319,A,NaN,2021-10-01 13:34:00
...,...,...,...,...,...,...,...,...,...,...,...
349,2021-10-05 19:55:00+00:00,56.32,56.33,56.28,56.280,9326.0,136.0,56.307352,L,NaN,2021-10-05 19:55:00
350,2021-10-05 19:56:00+00:00,56.28,56.30,56.28,56.290,6272.0,106.0,56.289332,L,NaN,2021-10-05 19:56:00
351,2021-10-05 19:57:00+00:00,56.30,56.33,56.30,56.320,8824.0,113.0,56.317979,L,NaN,2021-10-05 19:57:00
352,2021-10-05 19:58:00+00:00,56.31,56.33,56.27,56.285,17023.0,187.0,56.301397,L,NaN,2021-10-05 19:58:00


In [17]:
# Data Cleaning & Feature Engineering
# alpaca SDk automatically does data cleaning for NA parts in dataframe.

TECHNICAL_INDICATORS_LIST=['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']
data = DP.clean_data(data)
data = DP.add_technical_indicator(data, TECHNICAL_INDICATORS_LIST)
data = DP.add_vix(data)

The price of the first row for ticker  A  is NaN.  It will filled with the first valid price.
The price of the first row for ticker  L  is NaN.  It will filled with the first valid price.
Data clean finished!
Succesfully add technical indicators
Data before 2021-10-01T15:59:00-04:00 is successfully fetched
Data before 2021-10-02T15:59:00-04:00 is successfully fetched
Data before 2021-10-03T15:59:00-04:00 is successfully fetched
Data before 2021-10-04T15:59:00-04:00 is successfully fetched
Data before 2021-10-05T15:59:00-04:00 is successfully fetched
The price of the first row for ticker  VIXY  is NaN.  It will filled with the first valid price.
Data clean finished!


In [22]:
data.shape

(4259, 16)

In [18]:
# during preprocessing VIX is added for checking turbulence
data.head(100)

,time,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,VIXY
0,2021-10-01 09:30:00-04:00,157.48,157.48,157.48,157.48,0.0,A,0.0,NaN,NaN,NaN,NaN,NaN,157.48,157.48,22.17
1,2021-10-01 09:30:00-04:00,54.29,54.29,54.29,54.29,0.0,L,0.0,NaN,NaN,NaN,NaN,NaN,54.29,54.29,22.17
2,2021-10-01 09:31:00-04:00,157.48,157.48,157.48,157.48,0.0,A,0.0,157.48,157.48,NaN,NaN,NaN,157.48,157.48,22.17
3,2021-10-01 09:31:00-04:00,54.29,54.29,54.29,54.29,0.0,L,0.0,54.29,54.29,NaN,NaN,NaN,54.29,54.29,22.17
4,2021-10-01 09:32:00-04:00,157.48,157.48,157.48,157.48,0.0,A,0.0,157.48,157.48,NaN,NaN,NaN,157.48,157.48,22.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-10-01 10:17:00-04:00,54.29,54.29,54.29,54.29,0.0,L,0.0,54.29,54.29,NaN,22.222222,NaN,54.29,54.29,22.17
96,2021-10-01 10:18:00-04:00,157.48,157.48,157.48,157.48,0.0,A,0.0,157.48,157.48,NaN,0.000000,NaN,157.48,157.48,22.17
97,2021-10-01 10:18:00-04:00,54.29,54.29,54.29,54.29,0.0,L,0.0,54.29,54.29,NaN,0.000000,NaN,54.29,54.29,22.17
98,2021-10-01 10:19:00-04:00,157.48,157.48,157.48,157.48,0.0,A,0.0,157.48,157.48,NaN,0.000000,NaN,157.48,157.48,22.17


In [25]:
# converting data to numpy array
price_array, tech_array, turbulence_array = DP.df_to_array(data, if_vix='True')

ValueError: ignored

In [26]:
#  Train our DRL agent
#demo for elegantrl
ERL_PARAMS = {"learning_rate": 3e-5,
              "batch_size": 2048,
              "gamma": 0.99,
              "seed":312,
              "net_dimension":512}
train(start_date = '2021-10-11',
      end_date = '2021-10-15',
      ticker_list = ticker_list,
      data_source = 'alpaca',
      time_interval= '1Min',
      technical_indicator_list= TECHNICAL_INDICATORS_LIST,
      drl_lib='elegantrl',
      env=env,
      model_name='ppo',
      API_KEY = API_KEY,
      API_SECRET = API_SECRET,
      APCA_API_BASE_URL = APCA_API_BASE_URL,
      erl_params=ERL_PARAMS,
      cwd='./papertrading_erl',
      total_timesteps=1e5)


NameError: ignored

In [ ]:
# Test our DRL agent
account_value_erl=
 test(start_date = '2021-10-18',
      end_date = '2021-10-19',
      ticker_list = ticker_list,
      data_source = 'alpaca',
      time_interval= '1Min',
      technical_indicator_list= TECHNICAL_INDICATORS_LIST,
      drl_lib='elegantrl',
      env=env,
      model_name='ppo',
      API_KEY = API_KEY,
      API_SECRET = API_SECRET,
      APCA_API_BASE_URL = APCA_API_BASE_URL,
      cwd='./papertrading_erl',
      net_dimension = 512)

In [ ]:
#  Deploy our DRL Agent to Alpaca Paper Trading API
# We load the trained DRL model, and connect it to AlpacaPaperTrading environment to start shooting trades.
#demo for elegantrl
paper_trading_erl = 
AlpacaPaperTrading(ticker_list = DOW_30_TICKER,
time_interval = '1Min', 
drl_lib = 'elegantrl', 
agent = 'ppo',
cwd = './papertrading_erl', 
net_dim = 512,
state_dim = state_dim, 
action_dim= action_dim, 
API_KEY = API_KEY,
API_SECRET = API_SECRET,
APCA_API_BASE_URL = APCA_API_BASE_URL,
tech_indicator_list = TECHNICAL_INDICATORS_LIST,
turbulence_thresh=30, 
max_stock=1e2)
paper_trading_erl.run()